In [15]:
from tqdm.notebook import tqdm
from pprint import pprint
from glob import glob
import random
import spacy
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import requests

In [ ]:
! unzip wiki_data

In [31]:
documents = []

for fname in tqdm(glob('wiki_data/texts*.txt')):
    with open(fname) as f:
        document = ""
        for line in f:
            document = document + line.strip()
        
        documents.append(document)

len(documents), documents[0]

  0%|          | 0/1047 [00:00<?, ?it/s]

(1047,
 '= Carmichael number =In number theory, a Carmichael number is a composite numbern{\\displaystyle n}which satisfies the modular arithmetic congruence relation:bn−1≡1(modn){\\displaystyle b^{n-1}\\equiv 1{\\pmod {n}}}for all integersb{\\displaystyle b}which are relatively prime ton{\\displaystyle n}.They are named for Robert Carmichael.The Carmichael numbers are the subset K1 of the Knödel numbers.Equivalently, a Carmichael number is a composite numbern{\\displaystyle n}for whichbn≡b(modn){\\displaystyle b^{n}\\equiv b{\\pmod {n}}}for all integersb{\\displaystyle b}.OverviewFermat\'s little theorem states that if p is a prime number, then for any integer b, the number bp − b is an integer multiple of p.  Carmichael numbers are composite numbers which have this property.  Carmichael numbers are also called Fermat pseudoprimes or absolute Fermat pseudoprimes. A Carmichael number will pass a Fermat primality test to every base b relatively prime to the number, even though it is not

Training the model

In [32]:
nlp = spacy.load("en", disable=["parser", "ner", "tagger"])

def spacy_tokenizer(text):
    return [t.lemma_ for t in nlp(text)]

vec = TfidfVectorizer(tokenizer=spacy_tokenizer)
trained_vectors = vec.fit_transform(documents).todense()
texts = [[document, vector] for document, vector in zip(documents, trained_vectors)]

In [36]:
all_words = vec.vocabulary_

Initializing the index

In [33]:
answer = dict()

Searching

In [34]:
def lcp(a: str, b:str):
    '''
    Computes the longest common prefix of the strings `a` and `b`
    '''
    lcp = 0
    while lcp < len(a) and lcp < len(b) and a[lcp] == b[lcp]:
        lcp += 1
    return lcp

def most_similar(text: str):
    '''
    Returns the most similar word to `text` in the training vocabulary 
    by using Edit Distance* (ED from now on) and Longest Common Prefix
    (LCP from now).

    In general:
    - the lower the ED, the more similar the words are.
    - the greater the LCP, the more similar the words are.

    Assumption:
    - Typos are more likely to happen in the middle and the end of the words.
    That's why the LCP plays a major role in comparing the similarity of words.

    Final Similarity criteria**:
    - the similarity of two words `a` and `b` is ED(a, b) / exp(LCP(a, b))
    

    * Edit Distance is also called Levenshtein Distance
    ** It might change later
    '''

    result = [10**9, None] # [similarity, resulting_word]
    
    for word in all_words:
        result = min(result, [nltk.edit_distance(word, text) / np.exp(lcp(word, text)), word])
        
    return result[1]

def do_search(text: str):
    text = text.lower()
    if text not in all_words:
        text = most_similar(text)
    if text not in answer: # indexing the result for this text
        new_vector = vec.transform([text]).todense()
        texts_tmp = [[cosine(vector, new_vector), document] for document, vector in texts]
        texts_tmp.sort()
        answer[text] = [document for vector, document in texts_tmp[:10]]

    return answer[text]

In [37]:
do_search('Dijtra') # typo on purpose (Dijtra instead of Dijkstra)

['= Dijkstra\'s algorithm =Dijkstra\'s algorithm ( DYKE-strəz) is an algorithm for finding the shortest paths between nodes in a graph, which may represent, for example, road networks.  It was conceived by computer scientist Edsger W. Dijkstra in 1956 and published three years later.The algorithm exists in many variants. Dijkstra\'s original algorithm found the shortest path between two given nodes, but a more common variant fixes a single node as the "source" node and finds shortest paths from the source to all other nodes in the graph, producing a shortest-path tree.For a given source node in the graph, the algorithm finds the shortest path between that node and every other.:\u200a196–206\u200a It can also be used for finding the shortest paths from a single node to a single destination node by stopping the algorithm once the shortest path to the destination node has been determined. For example, if the nodes of the graph represent cities and edge path costs represent driving distanc

In [38]:
do_search('Belman') # typo on purpose (Belman instead of Bellman)

['= RSA (cryptosystem) =RSA (Rivest–Shamir–Adleman) is a public-key cryptosystem that is widely used for secure data transmission.  It is also one of the oldest.  The acronym "RSA" comes from the surnames of Ron Rivest, Adi Shamir and Leonard Adleman, who publicly described the algorithm in 1977.  An equivalent system was developed secretly in 1973 at GCHQ (the British signals intelligence agency) by the English mathematician Clifford Cocks.  That system was declassified in 1997.In a public-key cryptosystem, the encryption key is public and distinct from the decryption key, which is kept secret (private).An RSA user creates and publishes a public key based on two large prime numbers, along with an auxiliary value.  The prime numbers are kept secret.  Messages can be encrypted by anyone, via the public key, but can only be decoded by someone who knows the prime numbers.The security of RSA relies on the practical difficulty of factoring the product of two large prime numbers, the "factor

In [41]:
s = input()
do_search(s)

Spanning Tree


['= Kruskal\'s algorithm =Kruskal\'s algorithm finds a minimum spanning forest of an undirected edge-weighted graph. If the graph is connected, it finds a minimum spanning tree. (A minimum spanning tree of a connected graph is a subset of the edges that forms a tree that includes every vertex, where the sum of the weights of all the edges in the tree is minimized. For a disconnected graph, a minimum spanning forest is composed of a minimum spanning tree for each connected component.) It is a greedy algorithm in graph theory as in each step it adds the next lowest-weight edge that will not form a cycle to the minimum spanning forest.This algorithm first appeared in Proceedings of the American Mathematical Society, pp. 48–50 in 1956, and was written by Joseph Kruskal.Other algorithms for this problem include Prim\'s algorithm, the reverse-delete algorithm, and Borůvka\'s algorithm.Algorithmcreate a forest F (a set of trees), where each vertex in the graph is a separate treecreate a set S